In [9]:
%matplotlib inline

import numpy as np
import netCDF4 as nc
import xarray as xr
xr.set_options(display_style='html')
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# How to create a Dataset from scratch

In [11]:
def exponential(x, a, xc):
        return np.exp((x - xc) / a)
    
x = np.arange(-5, 5, 0.1)
t = np.arange(-5, 5, 0.1)
X, T = np.meshgrid(x, t)
Z1 = np.random.uniform(low=-5, high=5, size=X.shape)
Z2 = exponential(Z1, 3, X)
Z3 = exponential(Z1, 1, -X)


ds = xr.Dataset(
    data_vars=dict(
        var1=(["t", "x"], Z1), var2=(["t", "x"], Z2), var3=(["t", "x"], Z3)
    ),coords={"t": t, "x": x},)
display(ds)
#ds.var1.plot(); plt.show(); ds.var2.plot(); plt.show(); ds.var3.plot(); plt.show()

<xarray.Dataset>
Dimensions:  (t: 100, x: 100)
Coordinates:
  * t        (t) float64 -5.0 -4.9 -4.8 -4.7 -4.6 -4.5 ... 4.5 4.6 4.7 4.8 4.9
  * x        (x) float64 -5.0 -4.9 -4.8 -4.7 -4.6 -4.5 ... 4.5 4.6 4.7 4.8 4.9
Data variables:
    var1     (t, x) float64 -1.294 3.633 4.053 2.886 ... 3.995 1.792 -0.7752
    var2     (t, x) float64 3.44 17.19 19.12 12.54 ... 0.7907 0.3669 0.1508
    var3     (t, x) float64 0.001847 0.2816 0.4736 ... 5.975e+03 729.5 61.85

## Various notes on curvefit

In [ ]:
def fit(self):
    """
    Fits the currently selected data. Only used when selected, otherwise fitted at load time and saved to file
    :return: curve fit results
    """
    if self.selected:
        output = self.ds["ds1"].sel(Orientation=self.orientation, wavelength=self.wavelength, power=self.power).sel(
            x=slice(self.x0, self.x1), y=slice(self.y0, self.y1))
    else:
        output = self.ds["ds1"].sel(Orientation=self.orientation, wavelength=self.wavelength, power=self.power)
    
    curvefit = output.curvefit(["Polarization"], function, reduce_dims=["x", "y"],
                               param_names=["delta", "A", "B", "theta", "C"])
    curvefit_coefficients = curvefit.curvefit_coefficients  # idk what to do with the covars
    return curvefit_coefficients.values
    

fitted = function(theta_radians, *self.fit())

In [50]:
from cf_xarray.datasets import airds

def line(time, slope):
    t = (time - time[0]).astype(float)
    return slope * t

airds.air

ModuleNotFoundError: No module named 'cf_xarray'

In [ ]:
actual = airds.air.cf.isel(lat=4, lon=5).curvefit(coords=("time",), func=line)
expected = airds.air.cf.isel(lat=4, lon=5).cf.curvefit(coords="T", func=line)
assert_identical(expected, actual)

def plane(coords, slopex, slopey):
    x, y = coords
    return slopex * (x - x.mean()) + slopey * (y - y.mean())

actual = airds.air.isel(time=0).curvefit(coords=("lat", "lon"), func=plane)
expected = airds.air.isel(time=0).cf.curvefit(
    coords=("latitude", "longitude"), func=plane
)
assert_identical(expected, actual)

In [33]:
ds.curvefit?